In [1]:
'''script to regrid CMIP6 datatsets to target grid and store them'''

import numpy as np
import xarray as xr
import dask
import intake
import pandas as pd
import os
from collections import defaultdict
from tqdm.autonotebook import tqdm
from xmip.utils import google_cmip_col
from xmip.postprocessing import combine_datasets, _match_datasets,_concat_sorted_time
from cmip_catalogue_operations import reduce_cat_to_max_num_realizations, drop_vars_from_cat, drop_older_versions
from cmip_ds_dict_operations import select_period, pr_flux_to_m, drop_duplicate_timesteps, drop_coords, drop_incomplete
import xesmf as xe
import gcsfs
fs = gcsfs.GCSFileSystem() #list stores, stripp zarr from filename, load 

/tmp/ipykernel_1161/2346015076.py:10: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [17]:
fs.ls('leap-persistent/timh37/CMIP6/timeseries_eu_1p5/pr/ACCESS-CM2')

['leap-persistent/timh37/CMIP6/timeseries_eu_1p5/pr/ACCESS-CM2/ACCESS-CM2.gn.day.r10i1p1f1.pr.hist_ssp585',
 'leap-persistent/timh37/CMIP6/timeseries_eu_1p5/pr/ACCESS-CM2/ACCESS-CM2.gn.day.r1i1p1f1.pr.hist_ssp245',
 'leap-persistent/timh37/CMIP6/timeseries_eu_1p5/pr/ACCESS-CM2/ACCESS-CM2.gn.day.r2i1p1f1.pr.hist_ssp245',
 'leap-persistent/timh37/CMIP6/timeseries_eu_1p5/pr/ACCESS-CM2/ACCESS-CM2.gn.day.r2i1p1f1.pr.hist_ssp585',
 'leap-persistent/timh37/CMIP6/timeseries_eu_1p5/pr/ACCESS-CM2/ACCESS-CM2.gn.day.r3i1p1f1.pr.hist_ssp245',
 'leap-persistent/timh37/CMIP6/timeseries_eu_1p5/pr/ACCESS-CM2/ACCESS-CM2.gn.day.r3i1p1f1.pr.hist_ssp585',
 'leap-persistent/timh37/CMIP6/timeseries_eu_1p5/pr/ACCESS-CM2/ACCESS-CM2.gn.day.r4i1p1f1.pr.hist_ssp245',
 'leap-persistent/timh37/CMIP6/timeseries_eu_1p5/pr/ACCESS-CM2/ACCESS-CM2.gn.day.r4i1p1f1.pr.hist_ssp585',
 'leap-persistent/timh37/CMIP6/timeseries_eu_1p5/pr/ACCESS-CM2/ACCESS-CM2.gn.day.r5i1p1f1.pr.hist_ssp245',
 'leap-persistent/timh37/CMIP6/times

In [2]:
#configure settings
output_path = 'gs://leap-persistent/timh37/CMIP6/timeseries_eu_1p5/'
overwrite_existing = False #whether or not to process files for which output already exists (to-do: implement)

target_grid = xr.Dataset( #grid to interpolate CMIP6 simulations to
        {   "longitude": (["longitude"], np.arange(-30,22.5,1.5), {"units": "degrees_east"}),
            "latitude": (["latitude"], np.arange(70,30,-1.5), {"units": "degrees_north"}),})

query_vars = ['sfcWind','pr','psl'] #variables to process
required_vars = ['sfcWind','pr','psl'] #variables that includes models should provide

ssps = ['ssp245','ssp585']

In [3]:
#query simulations & manipulate data catalogue:
col = google_cmip_col() #google cloud catalogue
qc_col = intake.open_esm_datastore("https://storage.googleapis.com/leap-persistent-ro/data-library/catalogs/cmip6-test/leap-pangeo-cmip6-test.json") #temporary pangeo-leap-forge catalogue
noqc_col = intake.open_esm_datastore("https://storage.googleapis.com/leap-persistent-ro/data-library/catalogs/cmip6-test/leap-pangeo-cmip6-noqc-test.json")

col_df = col.df
qc_df = qc_col.df
nonqc_df = noqc_col.df

col_df['prio'] = 2
qc_df['prio'] = 3
nonqc_df['prio'] = 1

col.esmcat._df = pd.concat([col_df,qc_df,nonqc_df],ignore_index=True) #merge these catalogues
ssp_cats = defaultdict(dict)

#search catalogue per ssp (need to do this for each SSP separately as availability may differ between them)
for s,ssp in enumerate(ssps):
    ssp_cat = col.search( #find instances providing all required query_vars for both historical & ssp experiments
    experiment_id=['historical',ssp],
    table_id='day',
    variable_id=required_vars,
    require_all_on=['source_id', 'member_id','grid_label'])
    ssp_cats[ssp] = ssp_cat
    
ssp_cats_merged = ssp_cats[ssp] #merge catalogues for all ssps, and drop duplicate historical simulations
ssp_cats_merged.esmcat._df = pd.concat([v.df for k,v in ssp_cats.items()],ignore_index=True).drop_duplicates(ignore_index=True)
ssp_cats_merged = drop_older_versions(ssp_cats_merged) #if google cloud and leap-pangeo catalogues provide duplicate datasets, keep the newest version, and if the versions are identical, keep the leap-pangeo dataset
ssp_cats_merged = reduce_cat_to_max_num_realizations(ssp_cats_merged) #per model, select grid and 'ipf' combination providing most realizations (needs to be applied to both SSPs together to ensure the same variants are used under both scenarios)

In [16]:
for s,ssp in tqdm(enumerate(ssps)): #for each ssp:  
    #select historical and ssp data in merged catalogue for this particular ssp
    cat_to_open = ssp_cats_merged.search(
    experiment_id=['historical',ssp],
    table_id='day',
    variable_id=required_vars,
    source_id='ACCESS-CM2',
    require_all_on=['source_id', 'member_id','grid_label'])

    cat_to_open = drop_vars_from_cat(cat_to_open,[k for k in required_vars if k not in query_vars]) #out of required variables only process query variables
    #open datasets into dictionary
    cat_to_open.esmcat.aggregation_control.groupby_attrs = [] #to circumvent aggregate=false bug

    #to avoid this issue: https://github.com/intake/intake-esm/issues/496
        #doesn't actually aggregate if we set cmip6_cat.esmcat.aggregation_control.groupby_attrs = []
    kwargs = {'zarr_kwargs':{'consolidated':True,'use_cftime':True},'aggregate':True} #keyword arguments for generating dictionary of datasets from cmip6 catalogue
    ddict = cat_to_open.to_dataset_dict(**kwargs) #open datasets into dictionary

    #preprocess datasets in dictionary
    ddict = pr_flux_to_m(ddict) #convert pr flux to accumulated pr
    ddict = drop_duplicate_timesteps(ddict) #remove duplicate timesteps if datasets have them
    #ddict = select_period(ddict,1850,2100) #preselect time periods, do this at later stage in the chain?
    ddict = drop_coords(ddict,['bnds','nbnd','height']) #remove some unused auxiliary coordinates
    
    #concatenate historical and ssp datasets in time
    with dask.config.set(**{'array.slicing.split_large_chunks': True}):
        hist_ssp = combine_datasets(ddict,_concat_sorted_time,match_attrs =['source_id', 'grid_label','table_id','variant_label','variable_id'],combine_func_kwargs={'join':'inner','coords':'minimal'})    

    hist_ssp_ = defaultdict(dict) #probably a better way to do this, but there are approx. 1 files for which the time units are inconsistent between historical and ssp
    for k,v in hist_ssp.items():
        if v.time[-1].values.dtype != v.time[0].values.dtype:
            print('dropping ' + k +' due to inconsistent timestamps in historical and ssp runs')
            continue
        else:
            hist_ssp_[k] = v
            
    hist_ssp_ = drop_duplicate_timesteps(hist_ssp_) #remove overlap between historical and ssp experiments which sometimes exists
    hist_ssp_complete = drop_incomplete(hist_ssp_) #remove historical+ssp timeseries which are not montonically increasing or have large timegaps (based on Julius Buseckes rudimentary testing in CMIP6-LEAP-feadstock)
    
    #print(set(list(hist_ssp_.keys())) ^ set(list(hist_ssp_complete.keys())))
    #regrid these datasets to the target grid
    regridded_datasets = defaultdict(dict)
    for key,ds in tqdm(hist_ssp_complete.items()):
        variable = key.split('.')[-1]
        #check if dataset was already stored
        output_fn = os.path.join(output_path,variable,ds.source_id,key+'.hist_'+ssp) #output filename
        try:
            if (~overwrite_existing) & (output_fn.replace('gs://','') in fs.ls(os.path.join(output_path,variable,ds.source_id))):
                continue
        except:
            pass
            
        ds.attrs["time_concat_key"] = key #add current key information to attributes
        ds = ds.isel(dcpp_init_year=0,drop=True) #remove this coordinate
        
        regridder = xe.Regridder(ds,target_grid,'bilinear',ignore_degenerate=True,periodic=True) #create regridder for this dataset
        try:
            regridded_ds = regridder(ds,keep_attrs=True) #apply regridder
        except: #issue with 1 dataset that is chunked along two dimensions, rechunk that
            regridded_ds = regridder(ds.chunk({'time':100,'lat':1000,'lon':1000}),keep_attrs=True)
        
        regridded_datasets[key] = regridded_ds
        
        #store dataset as .zarr to leap-persistent storage
        try:
            regridded_ds.to_zarr(output_fn,mode='w') #fails if chunks are not uniform due to time concatenation
        except:
            regridded_ds[variable] = regridded_ds[variable].chunk({'time':'auto'})
            regridded_ds.to_zarr(output_fn,mode='w')
            
        regridded_ds.close()
        

0it [00:00, ?it/s]


--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.member_id.table_id.variable_id.grid_label.zstore.dcpp_init_year.version.prio'


  0%|          | 0/15 [00:00<?, ?it/s]


--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.member_id.table_id.variable_id.grid_label.zstore.dcpp_init_year.version.prio'


  0%|          | 0/20 [00:00<?, ?it/s]

In [21]:
np.unique(hist_ssp_['ACCESS-CM2.gn.day.r1i1p1f1.pr'].time.dt.year)

array([1850, 1851, 1852, 1853, 1854, 1855, 1856, 1857, 1858, 1859, 1860,
       1861, 1862, 1863, 1864, 1865, 1866, 1867, 1868, 1869, 1870, 1871,
       1872, 1873, 1874, 1875, 1876, 1877, 1878, 1879, 1880, 1881, 1882,
       1883, 1884, 1885, 1886, 1887, 1888, 1889, 1890, 1891, 1892, 1893,
       1894, 1895, 1896, 1897, 1898, 1899, 1900, 1901, 1902, 1903, 1904,
       1905, 1906, 1907, 1908, 1909, 1910, 1911, 1912, 1913, 1914, 1915,
       1916, 1917, 1918, 1919, 1920, 1921, 1922, 1923, 1924, 1925, 1926,
       1927, 1928, 1929, 1930, 1931, 1932, 1933, 1934, 1935, 1936, 1937,
       1938, 1939, 1940, 1941, 1942, 1943, 1944, 1945, 1946, 1947, 1948,
       1949, 1950, 1951, 1952, 1953, 1954, 1955, 1956, 1957, 1958, 1959,
       1960, 1961, 1962, 1963, 1964, 1965, 1966, 1967, 1968, 1969, 1970,
       1971, 1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981,
       1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992,
       1993, 1994, 1995, 1996, 1997, 1998, 1999, 20

Calculate total size of datasets

In [ ]:
x=0
for k,v in ddict.items():
    if 'ssp245' in k:
        x += v.nbytes/1000000000
x

List available members per model

In [8]:
models = ssp_cats_merged.df.source_id.unique()
count_members = np.zeros(len(models))

for k,ds in hist_ssp_complete.items():
    count_members[np.where(models==ds.source_id)[0][0]] = count_members[np.where(models==ds.source_id)[0][0]] +1

In [9]:
print(models)
np.floor_divide(count_members,3)

['ACCESS-CM2' 'ACCESS-ESM1-5' 'CESM2' 'CESM2-WACCM' 'CMCC-CM2-SR5'
 'CMCC-ESM2' 'CanESM5' 'EC-Earth3' 'EC-Earth3-Veg' 'FGOALS-g3' 'GFDL-CM4'
 'GFDL-ESM4' 'HadGEM3-GC31-LL' 'HadGEM3-GC31-MM' 'IITM-ESM' 'INM-CM4-8'
 'INM-CM5-0' 'IPSL-CM6A-LR' 'KACE-1-0-G' 'MIROC-ES2L' 'MIROC6'
 'MPI-ESM1-2-HR' 'MPI-ESM1-2-LR' 'MRI-ESM2-0' 'NorESM2-LM' 'NorESM2-MM'
 'TaiESM1' 'UKESM1-0-LL']


array([ 5., 39.,  2.,  3.,  1.,  1., 25., 61.,  1.,  1.,  1.,  1.,  5.,
        0.,  1.,  1.,  1., 11.,  3., 10., 45.,  2., 24.,  3.,  3.,  2.,
        1.,  5.])